In [ ]:
import torch
from pathlib import Path
from ultralytics import YOLO
from IPython.display import display
from PIL import Image

def validate_yolo_model():
    model_path = Path("runs/detect/train/weights/best.pt")
    images_dir = Path("images_casset/train")

    # Ensure the model exists
    if not model_path.exists():
        print(f"Error: Model file '{model_path}' not found.")
        return

    # Ensure images directory exists
    if not images_dir.exists() or not images_dir.is_dir():
        print(f"Error: Image directory '{images_dir}' not found.")
        return

    # Choose device: Prefer GPU but fall back to CPU if memory is low
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load YOLO model
    model = YOLO(model_path).to(device)

    # Get all image files (jpg and png)
    image_files = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png"))

    if not image_files:
        print("No image files found in the directory.")
        return

    print("Running detection...")

    # Free GPU memory before running (optional, but helps)
    if device == "cuda":
        torch.cuda.empty_cache()

    # Run detection with low-memory settings
    results = model.predict(
        image_files, 
        device="cpu",  # Force CPU to avoid GPU memory overflow
        batch=1,        # Process images one at a time
        half=False      # Avoid mixed precision issues
    )

    print("Detection complete. Displaying results:")

    # Display results in memory
    for result in results:
        img_with_boxes = result.plot()  # Get image with bounding boxes
        img_pil = Image.fromarray(img_with_boxes)  # Convert to PIL image
        display(img_pil)  # Show image in Jupyter Notebook

# Run the function
validate_yolo_model()
